# Import Sections

In [2]:
import os
import random

import openai
import pandas as pd
from datasets import load_dataset

import dspy
from dspy.evaluate import Evaluate
from dspy.teleprompt import BootstrapFewShotWithRandomSearch

# Downloading & Exploring the Dataset
The dataset can be seen/understood from here: [Dataset Page](https://huggingface.co/datasets/Divyanshu/indicxnli)

In [10]:
# Load Hindi(hi) Dataset. If it's first time, dataset will get downloaded at the location specified against "cache_dir" variable
dataset = load_dataset("Divyanshu/indicxnli", "hi", cache_dir=os.path.join(os.getcwd(), "datasets"))

<b>The dataset has three splits(as usual):</b>

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['premise', 'hypothesis', 'label'],
        num_rows: 392702
    })
    test: Dataset({
        features: ['premise', 'hypothesis', 'label'],
        num_rows: 5010
    })
    validation: Dataset({
        features: ['premise', 'hypothesis', 'label'],
        num_rows: 2490
    })
})

<b>Viewing one datapoint</b>

In [15]:
split = random.choice(['train', 'test', 'validation'])
index = random.randint(0, 2490)

dataset[split][index]

{'premise': 'और बीस साल पहले देखते हैं मुझे लगता है कि हम अभी क्या वे उह यौन क्रांति जहां uh गोली और uh uh के बाद कह रहे थे में मिलने के लिए शुरू कर रहे थे',
 'hypothesis': 'जब 1970 के दशक में सेक्स रिवोल्यूशन शुरू हुआ तो मैंने गोली खानी शुरू कर दी।',
 'label': 1}

<b>Premise</b>:  This is the initial statement or piece of text that provides factual information or context. It serves as the foundation or given information that we assume to be true for the inference task.

<b>Hypothesis</b>: This is a statement that may or may not follow logically from the premise. It's the claim we want to test against the premise to determine the logical relationship.

<b>Label</b>: This indicates the relationship between the premise and hypothesis:

Integer label `0` if hypothesis `entails` the premise, `2` if hypothesis `negates` the premise and `1` otherwise

## Code

#### Configuring LLM for DSPy

In [57]:
llm = dspy.LM('openai/gpt-4o-mini', api_key = os.getenv('OPENAI_API_KEY'))
dspy.configure(lm=llm)

---
Creating the `dataloader` function to load and return the datapoints as `dspy.Example` object.

#### dspy.Example
The core data type for data in DSPy is `Example`. `Example` represents items in training and test datasets. While doing evaluation & optimization runs, an individual datapoint will be of type `Example`.

`Example` objects have `with_inputs()` method that marks specific fields as inputs. The rest are labels or metadata.

In [41]:
def load_indicxlni(dataset, split="validation", nos_of_points=-1):
    if nos_of_points == -1:
        data_df = pd.DataFrame(dataset[split])
    else:
        data_df = pd.DataFrame(dataset[split]).head(nos_of_points)
    
    label_map = {
        0: "Yes",
        1: "Neutral",
        2: "No"
    }
    
    def as_dspy_example(row):
        return dspy.Example({
            "premise": row['premise'],
            "hypothesis": row['hypothesis'],
            "answer": label_map[row['label']]
        }).with_inputs("premise", "hypothesis")

    return list(data_df.apply(as_dspy_example, axis=1).values)

##### Train | Dev | Test Samples

In [43]:
all_train_set = load_indicxlni(dataset, "train")
all_dev_set = load_indicxlni(dataset, "validation")

random.seed(42)
random.shuffle(all_train_set)
random.shuffle(all_dev_set)

# 50 random train sample and 10 random dev sample
train_set, dev_set = all_train_set[: 50], all_dev_set[:10]

print("Size of training set: ", len(train_set))
print("Size of dev set: ", len(dev_set))

Size of training set:  50
Size of dev set:  10


In [44]:
all_test_set = load_indicxlni(dataset, "test")

random.shuffle(all_test_set)

test_set = all_test_set[:10]
print("Size of test set: ", len(test_set))

Size of test set:  10


---
#### dspy.Signature

`Signature` define the input-output behaviour of LLM calls in a declarative way. Let's define a signature for our dataset where `premise` and `hypothesis` would be inputs to the LLM or prompt and `answer` would be returned by the LLM.

The `doc_string` serves as the prompt to be sent to the LLM with the variables defined in the class as inputs and outputs.

In [46]:
class IndicXLNISignature(dspy.Signature):
    ("""You are given a premise and a hypothesis."""
    """You must indicate with Yes/No/Neutral answer whether we can logically conclude the hypothesis from the premise.""")

    premise = dspy.InputField()
    hypothesis = dspy.InputField()
    answer = dspy.OutputField(desc="Yes or No or Neutral")

Let's use a basic `dspy.Module` to see the prompt created with the above `Signature` class.

In [65]:
basic_prediction = dspy.Predict(IndicXLNISignature)

prediction = basic_prediction(premise = dataset['train'][2]['premise'], hypothesis = dataset['train'][2]['hypothesis'])
print(prediction)

Prediction(
    answer='Yes'
)


The prompt sent or used for the above prediction can be obtained and is this:

In [67]:
basic_prediction.inspect_history(n=1)





[2025-06-15T17:08:49.109041]

System message:

Your input fields are:
1. `premise` (str): 
2. `hypothesis` (str):
Your output fields are:
1. `answer` (str): Yes or No or Neutral
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## premise ## ]]
{premise}

[[ ## hypothesis ## ]]
{hypothesis}

[[ ## answer ## ]]
{answer}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        You are given a premise and a hypothesis.You must indicate with Yes/No/Neutral answer whether we can logically conclude the hypothesis from the premise.


User message:

[[ ## premise ## ]]
हमारी संख्या में से एक आपके निर्देशों का बारीकी से पालन करेगी।

[[ ## hypothesis ## ]]
मेरी टीम का एक सदस्य आपके आदेशों को बहुत सटीकता के साथ निष्पादित करेगा।

Respond with the corresponding output fields, starting with the field `[[ ## answer ## ]]`, and then ending with the marker for `[[ ## completed ## ]]`.


Response:

[[ ## answer ## ]]
Yes

[[

---
#### dspy.Module

DSPy Module abstracts prompting technique. We've already seen `dspy.Predict()` module before which is for basic prompting.

We will use advanced prompting technique, Chain Of Thought (COT), for this project, and we will use `dspy.ChainOfThought()` module for it.

In [68]:
class IndicXLNICOT(dspy.Module):
    def __init__(self):
        super().__init__()
        self.generate_answer = dspy.ChainOfThought(IndicXLNISignature)

    def forward(self, premise, hypothesis):
        return self.generate_answer(premise=premise, hypothesis=hypothesis)

Let's see how different this prompt would be as compared to `dspy.Predict()` module.

In [69]:
cot_zeroshot = IndicXLNICOT()

In [70]:
cot_prediction = cot_zeroshot(premise = dataset['train'][2]['premise'], hypothesis = dataset['train'][2]['hypothesis'])
print(cot_prediction)

Prediction(
    reasoning='प्रस्तावना में कहा गया है कि "हमारी संख्या में से एक आपके निर्देशों का बारीकी से पालन करेगी," जिसका अर्थ है कि एक सदस्य आपके निर्देशों का पालन करेगा। जबकि परिकल्पना में कहा गया है कि "मेरी टीम का एक सदस्य आपके आदेशों को बहुत सटीकता के साथ निष्पादित करेगा," यह भी एक सदस्य के आदेशों का पालन करने की बात करता है। दोनों में एक सदस्य के निर्देशों का पालन करने की बात की गई है, लेकिन परिकल्पना में "बहुत सटीकता" का उल्लेख है, जो कि प्रस्तावना में नहीं है। इसलिए, हम यह नहीं कह सकते कि परिकल्पना पूरी तरह से प्रस्तावना से निष्कर्षित होती है।',
    answer='Neutral'
)


In [71]:
cot_zeroshot.inspect_history(n=1)





[2025-06-15T18:18:43.539758]

System message:

Your input fields are:
1. `premise` (str): 
2. `hypothesis` (str):
Your output fields are:
1. `reasoning` (str): 
2. `answer` (str): Yes or No or Neutral
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## premise ## ]]
{premise}

[[ ## hypothesis ## ]]
{hypothesis}

[[ ## reasoning ## ]]
{reasoning}

[[ ## answer ## ]]
{answer}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        You are given a premise and a hypothesis.You must indicate with Yes/No/Neutral answer whether we can logically conclude the hypothesis from the premise.


User message:

[[ ## premise ## ]]
हमारी संख्या में से एक आपके निर्देशों का बारीकी से पालन करेगी।

[[ ## hypothesis ## ]]
मेरी टीम का एक सदस्य आपके आदेशों को बहुत सटीकता के साथ निष्पादित करेगा।

Respond with the corresponding output fields, starting with the field `[[ ## reasoning ## ]]`, then `[[ ## answer ## ]]`, and then end

---
#### Metrics & Evaluator

In DSPy, metric is a function that will take examples from your data and the output of LLM and return a score that quantifies how good the output is.

For simple classification or short-form QA tasks, this could be just "accuracy", or "exact match" or "F1-Score". For long-form outputs, metric could be a smaller DSPy program that checks multiple properties of the output(using AI feedback from LLMs).

Simpler metrics are already defined in DSPY:
- `dspy.evaluate.metrics.answer_exact_match`
- `dspy.evaluate.metrics.answer_passage_match`

The metrics could be more complex, e.g. check for multiple properties. For that, write your own function.

In [72]:
indicxlni_accuracy = dspy.evaluate.metrics.answer_exact_match

Once metric is decided/defined, evaluations can be done either in a simple Python loop:

```
scores = []
for x in devset:
    pred = program(**x.inputs())
    score = metric(x, pred)
    scores.append(score)
```

or can use built-in utilites in `Evaluate` utilities.

In [77]:
evaluator = Evaluate(devset=test_set, num_threads=1, display_progress=True, display_table=len(test_set))

In [78]:
evaluator(cot_zeroshot, metric=indicxlni_accuracy)

Average Metric: 7.00 / 10 (70.0%): 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 743.37it/s]

2025/06/15 22:19:06 INFO dspy.evaluate.evaluate: Average Metric: 7 / 10 (70.0%)


,premise,hypothesis,example_answer,reasoning,pred_answer,answer_exact_match
0,सुरक्षा सुनिश्चित करने के लिए जनरेटरों को बंद करना पड़ा और लिफ्ट ब...,जनरेटर एक सुरक्षा जोखिम थे।,Yes,प्रस्तावना में कहा गया है कि सुरक्षा सुनिश्चित करने के लिए जनरेटरो...,Yes,✔️ [True]
1,"भगवान का उल्लेख केवल प्रकृति के भगवान के रूप में किया जाता है, जिस...",लोग अलग-अलग होंगे लेकिन समान होंगे।,Yes,प्रस्तावना में कहा गया है कि सभी लोगों को राष्ट्रों के समुदाय में ...,Yes,✔️ [True]
2,"अप्रैल १८६५ में उस व्यक्ति की हत्या के साथ, जिसने विचारों के एक नए...",इस हमले को रूस ने अंजाम दिया था।,Neutral,प्रस्तावना में यह कहा गया है कि एक व्यक्ति की हत्या अप्रैल 1865 मे...,No,
3,"दूसरे शब्दों में, क्या होता है कुछ की तरह है एक जादूगर के अब-तुम-अ...",जो हो रहा है वह हैरान करने वाला है।,Yes,"प्रस्तावना में एक जादूगर के कार्यों का उल्लेख किया गया है, जो आमतौ...",Neutral,
4,"आप जानते हैं, आप नहीं बच सकते, आप जीवित नहीं रह सकते अगर आपके पास ...",आपको 5000 फीट से अधिक काउंटर प्रेशर की जरूरत होती है।,Neutral,प्रस्तावना में कहा गया है कि ऊंचाइयों पर सांस लेने के लिए काउंटर प...,Neutral,✔️ [True]
5,"एक साधारण उदाहरण के रूप में, मान लीजिए कि काम साझा करने की लागत 10...",आप अनुमान लगा सकते हैं कि काम साझा करने की लागत मूल डाक से कम है।,Yes,प्रस्तावना में कहा गया है कि काम साझा करने की लागत 10a है और मूल ड...,Yes,✔️ [True]
6,आजादी के बाद यहां संसद बनाने की योजना शून्य हो गई।,इस स्थान के लिए संसद पर दृढ़ता से विचार किया गया था।,Neutral,प्रस्तावना में कहा गया है कि आजादी के बाद संसद बनाने की योजना शून्...,No,
7,हाँ मैं उसे सुन सकता हूँ,मैं उसे सुन नहीं सकता.,No,"The premise states ""हाँ मैं उसे सुन सकता हूँ,"" which translates to...",No,✔️ [True]
8,"इस समूह में 23वां संशोधन भी है, जो कोलंबिया जिले में अन्यथा योग्य ...",23वें संशोधन में कहा गया है कि अगर आप कैलिफोर्निया में रहते हैं तो...,No,23वें संशोधन का संदर्भ कोलंबिया जिले में अन्यथा योग्य नागरिकों को ...,No,✔️ [True]
9,"रिसॉर्ट के केंद्र में, भीतरी लैगून के आश्रय वाले पानी में, डॉल्फ़ि...",आप रिसॉर्ट में डोफिन के साथ तैर सकते हैं।,Yes,प्रस्तावना में कहा गया है कि रिसॉर्ट में डॉल्फ़िन कार्यक्रम के साथ...,Yes,✔️ [True]


70.0

---
### Optimization

In [79]:
bootstrap_optimizer = BootstrapFewShotWithRandomSearch(
    max_bootstrapped_demos=8,
    max_labeled_demos=8,
    num_candidate_programs=10,
    num_threads=8,
    metric=indicxlni_accuracy
)

Going to sample between 1 and 8 traces per predictor.
Will attempt to bootstrap 10 candidate sets.


In [80]:
cot_fewshot = bootstrap_optimizer.compile(cot_zeroshot, trainset=train_set, valset=dev_set)

Average Metric: 5.00 / 10 (50.0%): 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:04<00:00,  2.00it/s]

2025/06/15 22:30:40 INFO dspy.evaluate.evaluate: Average Metric: 5 / 10 (50.0%)



New best score: 50.0 for seed -3
Scores so far: [50.0]
Best score so far: 50.0
Average Metric: 5.00 / 10 (50.0%): 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:03<00:00,  3.10it/s]

2025/06/15 22:30:44 INFO dspy.evaluate.evaluate: Average Metric: 5 / 10 (50.0%)



Scores so far: [50.0, 50.0]
Best score so far: 50.0


...


Scores so far: [50.0, 50.0, 50.0, 50.0, 40.0, 50.0, 50.0, 60.0, 50.0, 50.0, 50.0, 50.0]
Best score so far: 60.0


 24%|█████████████████████████████████████████████████████████████████████████████▌                                                                                                                                                                                                                                                     | 12/50 [00:19<01:02,  1.65s/it]


Bootstrapped 8 full traces after 12 examples for up to 1 rounds, amounting to 12 attempts.
Average Metric: 4.00 / 10 (40.0%): 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:03<00:00,  3.06it/s]

2025/06/15 22:34:23 INFO dspy.evaluate.evaluate: Average Metric: 4 / 10 (40.0%)



Scores so far: [50.0, 50.0, 50.0, 50.0, 40.0, 50.0, 50.0, 60.0, 50.0, 50.0, 50.0, 50.0, 40.0]
Best score so far: 60.0
13 candidate programs found.


In [81]:
evaluator(cot_fewshot, metric=indicxlni_accuracy)

Average Metric: 7.00 / 10 (70.0%): 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:22<00:00,  2.22s/it]

2025/06/15 22:35:39 INFO dspy.evaluate.evaluate: Average Metric: 7 / 10 (70.0%)


,premise,hypothesis,example_answer,reasoning,pred_answer,answer_exact_match
0,सुरक्षा सुनिश्चित करने के लिए जनरेटरों को बंद करना पड़ा और लिफ्ट ब...,जनरेटर एक सुरक्षा जोखिम थे।,Yes,प्रस्तावना में कहा गया है कि सुरक्षा सुनिश्चित करने के लिए जनरेटरो...,Yes,✔️ [True]
1,"भगवान का उल्लेख केवल प्रकृति के भगवान के रूप में किया जाता है, जिस...",लोग अलग-अलग होंगे लेकिन समान होंगे।,Yes,प्रस्तावना में कहा गया है कि सभी लोगों को राष्ट्रों के समुदाय में ...,Yes,✔️ [True]
2,"अप्रैल १८६५ में उस व्यक्ति की हत्या के साथ, जिसने विचारों के एक नए...",इस हमले को रूस ने अंजाम दिया था।,Neutral,The premise discusses the assassination of a person in April 1865 ...,No,
3,"दूसरे शब्दों में, क्या होता है कुछ की तरह है एक जादूगर के अब-तुम-अ...",जो हो रहा है वह हैरान करने वाला है।,Yes,"प्रस्तावना में जादूगर के प्रदर्शन की तुलना की गई है, जिसमें कुछ ची...",Neutral,
4,"आप जानते हैं, आप नहीं बच सकते, आप जीवित नहीं रह सकते अगर आपके पास ...",आपको 5000 फीट से अधिक काउंटर प्रेशर की जरूरत होती है।,Neutral,The premise states that one cannot survive without counter pressur...,Neutral,✔️ [True]
5,"एक साधारण उदाहरण के रूप में, मान लीजिए कि काम साझा करने की लागत 10...",आप अनुमान लगा सकते हैं कि काम साझा करने की लागत मूल डाक से कम है।,Yes,प्रस्तावना में कहा गया है कि काम साझा करने की लागत 10a है और मूल ड...,Yes,✔️ [True]
6,आजादी के बाद यहां संसद बनाने की योजना शून्य हो गई।,इस स्थान के लिए संसद पर दृढ़ता से विचार किया गया था।,Neutral,प्रस्तावना में कहा गया है कि आजादी के बाद यहां संसद बनाने की योजना...,No,
7,हाँ मैं उसे सुन सकता हूँ,मैं उसे सुन नहीं सकता.,No,"The premise states ""हाँ मैं उसे सुन सकता हूँ,"" which translates to...",No,✔️ [True]
8,"इस समूह में 23वां संशोधन भी है, जो कोलंबिया जिले में अन्यथा योग्य ...",23वें संशोधन में कहा गया है कि अगर आप कैलिफोर्निया में रहते हैं तो...,No,प्रस्तावना में कहा गया है कि 23वां संशोधन कोलंबिया जिले में अन्यथा...,No,✔️ [True]
9,"रिसॉर्ट के केंद्र में, भीतरी लैगून के आश्रय वाले पानी में, डॉल्फ़ि...",आप रिसॉर्ट में डोफिन के साथ तैर सकते हैं।,Yes,The premise states that there is a swimming program with dolphins ...,Yes,✔️ [True]


70.0

In [82]:
cot_fewshot.inspect_history(n=1)





[2025-06-15T22:35:39.891624]

System message:

Your input fields are:
1. `premise` (str): 
2. `hypothesis` (str):
Your output fields are:
1. `reasoning` (str): 
2. `answer` (str): Yes or No or Neutral
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## premise ## ]]
{premise}

[[ ## hypothesis ## ]]
{hypothesis}

[[ ## reasoning ## ]]
{reasoning}

[[ ## answer ## ]]
{answer}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        You are given a premise and a hypothesis.You must indicate with Yes/No/Neutral answer whether we can logically conclude the hypothesis from the premise.


User message:

This is an example of the task, though some input or output fields are not supplied.

[[ ## premise ## ]]
एक परीक्षण में चिकित्सकों द्वारा किए गए हस्तक्षेप के परिणाम उतने ही अच्छे थे जितना कि दूसरे में सामाजिक कार्यकर्ताओं और मनोवैज्ञानिकों द्वारा किए गए हस्तक्षेप के।

[[ ## hypothesis ## ]]
ऐसे कोई पर्याप्त सबूत